In [20]:
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, year, month

In [4]:
spark = SparkSession \
        .builder \
        .appName("Using Row Number") \
        .getOrCreate()

24/09/12 10:55:24 WARN Utils: Your hostname, ken resolves to a loopback address: 127.0.1.1; using 192.168.15.13 instead (on interface wlp2s0)
24/09/12 10:55:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/12 10:55:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Using Row Number to Identify Duplicates

In [7]:
dados = [
(datetime(2024, 1, 1), 100.0, "12345678901"),
    (datetime(2024, 1, 15), 150.0, "12345678901"),
    (datetime(2024, 1, 1), 100.0, "12345678901"),
    (datetime(2024, 1, 20), 200.0, "98765432100"),
    (datetime(2024, 1, 20), 200.0, "98765432100"),
    (datetime(2024, 2, 5), 250.0, "22233344455"),
    (datetime(2024, 2, 15), 300.0, "33344455566"),
    (datetime(2024, 2, 5), 250.0, "22233344455"),
    (datetime(2024, 2, 25), 350.0, "44455566677"),
    (datetime(2024, 2, 25), 350.0, "44455566677"),
    (datetime(2024, 3, 10), 400.0, "55566677788"),
    (datetime(2024, 3, 20), 450.0, "66677788899"),
    (datetime(2024, 3, 10), 400.0, "55566677788"),
    (datetime(2024, 3, 25), 500.0, "77788899900"),
    (datetime(2024, 3, 30), 550.0, "88899900011"),
    (datetime(2024, 4, 1), 600.0, "99900011122"),
    (datetime(2024, 4, 10), 650.0, "00011122233"),
    (datetime(2024, 4, 10), 650.0, "00011122233"),
    (datetime(2024, 4, 20), 700.0, "11122233344"),
    (datetime(2024, 4, 25), 750.0, "22233344455"),
    (datetime(2024, 5, 5), 800.0, "33344455566"),
    (datetime(2024, 5, 10), 850.0, "44455566677"),
    (datetime(2024, 5, 10), 850.0, "44455566677"),
    (datetime(2024, 5, 15), 900.0, "55566677788"),
    (datetime(2024, 5, 20), 950.0, "66677788899")
]

colunas = ["data", "valor", "id"]

In [8]:
df = spark.createDataFrame(dados, colunas)

In [10]:
df.show(25,truncate=False)

+-------------------+-----+-----------+
|data               |valor|id         |
+-------------------+-----+-----------+
|2024-01-01 00:00:00|100.0|12345678901|
|2024-01-15 00:00:00|150.0|12345678901|
|2024-01-01 00:00:00|100.0|12345678901|
|2024-01-20 00:00:00|200.0|98765432100|
|2024-01-20 00:00:00|200.0|98765432100|
|2024-02-05 00:00:00|250.0|22233344455|
|2024-02-15 00:00:00|300.0|33344455566|
|2024-02-05 00:00:00|250.0|22233344455|
|2024-02-25 00:00:00|350.0|44455566677|
|2024-02-25 00:00:00|350.0|44455566677|
|2024-03-10 00:00:00|400.0|55566677788|
|2024-03-20 00:00:00|450.0|66677788899|
|2024-03-10 00:00:00|400.0|55566677788|
|2024-03-25 00:00:00|500.0|77788899900|
|2024-03-30 00:00:00|550.0|88899900011|
|2024-04-01 00:00:00|600.0|99900011122|
|2024-04-10 00:00:00|650.0|00011122233|
|2024-04-10 00:00:00|650.0|00011122233|
|2024-04-20 00:00:00|700.0|11122233344|
|2024-04-25 00:00:00|750.0|22233344455|
|2024-05-05 00:00:00|800.0|33344455566|
|2024-05-10 00:00:00|850.0|44455566677|


In [13]:
windowSpec = Window.partitionBy("data", "id").orderBy("valor")

df_with_row = df.withColumn("row_number", row_number().over(windowSpec))

df_no_duplicates = df_with_row.filter(df_with_row["row_number"] == 1)

df_no_duplicates.show(truncate=False)

+-------------------+-----+-----------+----------+
|data               |valor|id         |row_number|
+-------------------+-----+-----------+----------+
|2024-01-01 00:00:00|100.0|12345678901|1         |
|2024-01-15 00:00:00|150.0|12345678901|1         |
|2024-01-20 00:00:00|200.0|98765432100|1         |
|2024-02-05 00:00:00|250.0|22233344455|1         |
|2024-02-15 00:00:00|300.0|33344455566|1         |
|2024-02-25 00:00:00|350.0|44455566677|1         |
|2024-03-10 00:00:00|400.0|55566677788|1         |
|2024-03-20 00:00:00|450.0|66677788899|1         |
|2024-03-25 00:00:00|500.0|77788899900|1         |
|2024-03-30 00:00:00|550.0|88899900011|1         |
|2024-04-01 00:00:00|600.0|99900011122|1         |
|2024-04-10 00:00:00|650.0|00011122233|1         |
|2024-04-20 00:00:00|700.0|11122233344|1         |
|2024-04-25 00:00:00|750.0|22233344455|1         |
|2024-05-05 00:00:00|800.0|33344455566|1         |
|2024-05-10 00:00:00|850.0|44455566677|1         |
|2024-05-15 00:00:00|900.0|5556

In [26]:
df_d = df_no_duplicates \
        .groupBy(year("data").alias("ano"), month("data").alias("mes")) \
        .count() \
        .orderBy(["ano", "mes"])

df_d.show(truncate=False)

+----+---+-----+
|ano |mes|count|
+----+---+-----+
|2024|1  |3    |
|2024|2  |3    |
|2024|3  |4    |
|2024|4  |4    |
|2024|5  |4    |
+----+---+-----+

